In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time


cookies = {
    'll': '108288',
    'bid': 'OyQ1O5ICyw0',
    'douban-fav-remind': '1',
    '__gads': 'ID=ab683fe3b045ed9a:T=1699349101:RT=1699349101:S=ALNI_MZcN2z0-MDESupq8yh9Flzku6hrUA',
    '__gpi': 'UID=00000c8182eb6fdd:T=1699349101:RT=1699349101:S=ALNI_MYM-qX0uBlZgIxlsQ3TFJh-MZLdSQ',
    'cto_bundle': 'FNc8kl9CNXBJWXdwM0doN3FmZkdUYkVTMkd2aGdaSmNqVGhGNzE2ODhmV3E0TXRHRkkxVzBxJTJGYXpRQXZHb1hIRkhLTUN6WGlSTE9iRW94UVR4OVZtRyUyQlZtT24wSktzOG9XNTc0TUxGUHZnaFdvVms2NU9WNzRXdyUyQlB6QVVjdlVBQ3VWTGswUE9DVVM5a2NlemRRekMxek9DaUElM0QlM0Q',
    'push_noty_num': '0',
    'push_doumail_num': '0',
    '_pk_id.100001.4cf6': '488320e055e75cf6.1699365244.',
    '__yadk_uid': 'uVWHUdecRuBKlbE9bTBVeX6witchE2FO',
    'dbcl2': '251070069:7SJG+Ywsc60',
    '__utmv': '30149280.25107',
    '_vwo_uuid_v2': 'DF38D49F2BEE9D58868D9948053505AFE|894eb797db1d249245b848f301cad3f8',
    '_ga_Y4GN1R87RG': 'GS1.1.1699461573.7.1.1699461651.0.0.0',
    '_ga': 'GA1.2.1926706278.1699352846',
    'ck': '0twp',
    'ap_v': '0,6.0',
    '__utma': '30149280.1744815168.1699348021.1699460957.1700977802.7',
    '__utmc': '30149280',
    '__utmz': '30149280.1700977802.7.2.utmcsr=cn.bing.com|utmccn=(referral)|utmcmd=referral|utmcct=/',
    '__utmt': '1',
    '__utmb': '30149280.2.10.1700977802',
    'frodotk_db': '1a15394433b7c5464e3fabcc9b274fd2',
    '_pk_ref.100001.4cf6': '%5B%22%22%2C%22%22%2C1700977808%2C%22https%3A%2F%2Fwww.douban.com%2F%22%5D',
    '_pk_ses.100001.4cf6': '1',
}

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
    'Connection': 'keep-alive',
    'Referer': 'https://movie.douban.com/subject/4058939/reviews',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36 Edg/119.0.0.0',
    'sec-ch-ua': '"Microsoft Edge";v="119", "Chromium";v="119", "Not?A_Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}

In [ ]:
def getLongContent(id):
    response = requests.get(f'https://movie.douban.com/j/review/{id}/full', headers=headers, cookies=cookies)
    html = response.json()['html']
    content = BeautifulSoup(html,'html.parser').text
    return content

def getPage(movieId,page):
    params = (
        ('start', str(page * 20)),
    )

    response = requests.get(f'https://movie.douban.com/subject/{movieId}/reviews', headers=headers, params=params, cookies=cookies)
    html = response.text
    soup = BeautifulSoup(html,'html.parser')


    review_list = soup.find('div',{'class':'review-list'})
    review_items = review_list.find_all('div',{'class':'main review-item'})
    review_infos = []
    for item in review_items:
        id = item['id']
        main_hd = item.find('header',{'class':'main-hd'})
        
        user = main_hd.find('a',{'class':'name'}).text.strip()
        try:
            rating = main_hd.find('span',{'class':'main-title-rating'})['title']
        except:
            rating = None
        comment_time = main_hd.find('span',{'class':'main-meta'}).text.strip()

        main_bd = item.find('div',{'class':'main-bd'})
        title = main_bd.find('h2').text.strip()
        action = main_bd.find('div',{'class':'action'})
        usefulCount = action.find('a',{'title':'有用'}).text.strip()
        if not usefulCount:
            usefulCount = 0
        
        noUsefulCount = action.find('a',{'title':'没用'}).text.strip()
        if not noUsefulCount:
            noUsefulCount = 0
        replyCount = action.find('a',{'class':'reply cls_abnormal'}).text.strip().replace('回应','')
        longContent = getLongContent(id)
        info = {
            'User':user,
            'Comment Time':comment_time,
            'Useful Count':usefulCount,
            'Useless Count':noUsefulCount,
            'Reply Count':replyCount,
            'Rating':rating,
            'Review Title':title,
            'Review':longContent,
        }    
        print(info)
        time.sleep(1)
        review_infos.append(info)
    return review_infos

In [32]:
movieId = '4058939'
all_review_infos = []
for page in range(186):
    review_infos = getPage(movieId,page)
    all_review_infos += review_infos
    

{'User': 'Ambition', 'Comment Time': '2023-07-27 01:59:08', 'Useful Count': '1', 'Useless Count': 0, 'Reply Count': '0', 'Rating': '力荐', 'Review Title': '“女性从不该被某一种特质束缚”', 'Review': '看完芭比，我突然对女性主义这个概念有了更加深层次的理解，不仅对于女性，更包括对男性的理解和认识。当芭比和肯第一次进入真实世界，男凝也随之出现，火辣的身材会被上下打量，吹口哨，抛媚眼，甚至是对拍芭比的屁股，这些都是男凝视角下，女性是被怎样的对待。这些情况是最浅显的，却也是最典型且每个女生多多少少会经历的。而芭比乐园变成肯乐园的时候，女性在男凝视角下的形象就变得更加明显。超短裙，女仆装，永远掺杂着崇拜，随着男生活动而变化的视线以及一句“wow，you know so much”。不得不说这些看似正常甚至是一种调情手段的情况并不少见，甚至已经成为普罗大众眼中的一种刻板印象。男人主导下的世界，女性是无知而美丽的花瓶，她们有能力却愿意屈居男人之下，为了孩子的牺牲也变成了一种理所应当，甚至大多数女性仍然会下意识的遵循男性主导审美下去打扮自己。就像萨拉的妈妈说的那样“what about ordinary Barbie?”是啊，普通芭比应该是怎么样的呢？女性主义永远不会局限在特殊或者小部分女性身上，她应该是拥抱每一位女性的，事业型大女主和家庭主妇都平等的受女性主义的保护。她应该是尊重每一种生活方式的，我始终认为女性主义所追求的是一种融合了包容，尊重和欣赏的态度，而并非是极端追求某一种特定的对待，或者是性别对立。想做一个母亲那就去做一个母亲，无论是家庭主妇也好，全职妈妈也好，都不应该被划出女性主义的圈子。母亲创造了这个世界。你可以不勇敢，可以不漂亮，可以不聪明，可以不热情，女性从不应该被某一些特质困住。这部电影并非是挑起性别对立，她只是讲述了一个童话故事，而这足以为女性意识的觉醒的火焰添一把柴。'}
{'User': 'Julia', 'Comment Time': '2023-07-27 01:52:43', 'Useful Count': 0, 'Useless Count': 0, 'Reply Count': '0

In [33]:
df = pd.DataFrame(all_review_infos)
df = df.drop_duplicates()
df.to_excel('data.xlsx',index=False)